# COMP47670 Assignment 2 - Text Classification

#### Student Name: Dara Leonard
#### Student ID: 19202478

In [117]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

#### Task 1: Scrape all reviews for each category and store them as three separate datasets
For Task 1 I have chosen the Automotive, Cafes, Fashion categories as my data sources.


In [118]:
data_source = 'http://mlg.ucd.ie/modules/yalp/'
business_categories = ['automotive_list.html', 'cafes_list.html', 'fashion_list.html']

def extract_category_data(url):
    response = requests.get(url)
    data = response.text
    return BeautifulSoup(data,'html.parser')


def categorise_rating(review_stars):
    return 'positive' if int(review_stars.get('alt').split('-')[0]) >= 4 else 'negative'


def extract_reviews(url):
    soup = extract_category_data(url)
    reviews_block = soup.findAll("div", { "class" : "review" })
    review_list =[]
    review_info_columns = ['comment', 'rating']
    review_df = pd.DataFrame(columns=review_info_columns)

    for i, reviews in enumerate(reviews_block):
        review = {}
        review_stars = reviews.find('img')
        review["comments"] = reviews.find("p", { "class" : "review-text" }).get_text()
        review["rating"] = categorise_rating(review_stars)
        review_list.append(review)        
        
        review_tmp = dict({
            'comment': reviews.find("p", { "class" : "review-text" }).get_text(),
            'rating' : categorise_rating(review_stars)
        })
        review_df.loc[i] = review_tmp

    return review_list, review_df


def extract_data(url):
    soup = extract_category_data(url)
    links = soup.find_all('a')
    review_list =[]
    for link in links:
        url = data_source + link.get('href')
        review, review_df = extract_reviews(url)
        review_list.append(review_df)
    return pd.concat(review_list)


url = data_source + business_categories[0]

consolidated_automotive_reviews_df = extract_data(url)
consolidated_automotive_reviews_df.to_json('automotive.json', orient='records')

In [119]:
consolidated_automotive_reviews_df.head()

,comment,rating
0,The man that was working tonight (8-12-17) was...,negative
1,Chris is a very rude person. Gave me an attitu...,negative
2,One of my favorite gas station to stop at. The...,positive
3,Oh thank Heaven for Seven Eleven! I don't know...,negative
4,Five stars because of the guy who works weekda...,positive


In [120]:
consolidated_automotive_reviews_df.isnull().sum()

comment    0
rating     0
dtype: int64

In [121]:
consolidated_automotive_reviews_df.shape

(2000, 2)

In [122]:
consolidated_automotive_reviews_df.groupby('rating')['rating'].count()

rating
negative     788
positive    1212
Name: rating, dtype: int64

#### Task 2: For each of the chose categories; apply any necessary pre-processing, build a classification model & test said classification model

In [123]:
print('Done')

Done
